<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>Travelling Salesman Problem</strong></h1>
  <h3><strong>Ejercicio 6</strong></h3>
</div>

Implementar el modelo MTZ o el modelo de flujo en redes (o ambos) para el problema del TSP vistos en clase con la siguiente instancia del problema:

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### **Conjunto de índices y Parámetros**

Modelo del TSP con variables de flujo:  
Sean $x_{ij} = 1$, $i, j \in N : i \neq j$ las variables de decisión que indican que se visita la ciudad $j$ inmediatamente después de la ciudad $i$ y $x_{ij} = 0$ en otro caso. Asimismo, sean $y_{ij} \geq 0$, $i, j \in N : i \neq j$ las variables de decisión que denotan el flujo de la ciudad $i$ a la ciudad $j$. Se considera el parámetro $b_i$, $i \in N$, con $b_1 = n - 1$ y $b_i = -1$, $i = 2, \ldots, n$.

In [3]:
tij = [ #distancias para nuestro flujo de redes :)
    [0, 26, 82, 65, 100, 145, 100],
    [66, 0, 56, 39, 109, 156, 80],
    [43, 57, 0, 16, 53, 100, 128],
    [27, 41, 62, 0, 97, 144, 37],
    [109, 135, 161, 174, 0, 47, 41],
    [157, 161, 114, 130, 60, 0, 23],
    [30, 101, 123, 89, 75, 105, 0]
]

n = len(tij)
N = range(n) #conjunto de ciudades

### **Variables de decisión**

$$ y_{ij} \geq 0 \quad \forall i, j \in N : i \neq j $$


In [4]:
y = {} #flujo de ciudad i a ciudad j
for i in N:
    for j in N:
        if i != j: # no tiene sentido ir de una ciudad a la misma
            y[i, j] = solver.NumVar(0, solver.infinity(), f'y[{i},{j}]') 

$$ x_{ij} \in \{0, 1\} \quad \forall i, j \in N : i \neq j $$

In [5]:
x = {} #viaje inmediato de ciudad a ciudad
for i in N:
    for j in N:
        if i != j:
            x[i, j] = solver.BoolVar(f'x[{i},{j}]')

### **Función objetivo**

$$ \min \sum_{i \in N} \sum_{j \in N : i \neq j} t_{ij} x_{ij} $$

In [6]:
camino = solver.Objective()

camino = sum(tij[i][j] * x[i, j] for i in N for j in N if i != j) 

solver.Minimize(camino) #minimizamos el camino recorrido 

### **Restricciones**

$$ \sum_{j \in N : i \neq j} x_{ij} = 1 \quad \forall i \in N $$

In [7]:
for i in N:
    solver.Add(sum(x[i, j] for j in N if i != j) == 1)

$$ \sum_{i \in N : i \neq j} x_{ij} = 1 \quad \forall j \in N $$

In [8]:
for j in N:
    solver.Add(sum(x[i, j] for i in N if i != j) == 1)

$$ \sum_{j \in N : i \neq j} y_{ij} - \sum_{j \in N : i \neq j} y_{ji} = b_i \quad \forall i \in N $$

In [9]:
b = [n - 1] + [-1] * (n - 1) # b = [n - 1, -1, -1, ..., -1]
for i in N:
    solver.Add(sum(y[i, j] for j in N if i != j) - solver.Sum(y[j, i] for j in N if i != j) == b[i])


$$ y_{ij} \leq (n - 1) x_{ij} \quad \forall i, j \in N : i \neq j $$


In [10]:
for i in N:
    for j in N:
        if i != j:
            solver.Add(y[i, j] <= (n - 1) * x[i, j])

### **Resolver**

In [11]:
solver.Solve()

0

In [12]:
print('Función objetivo =', solver.Objective().Value())

Función objetivo = 280.0


In [13]:
for i in N:
    for j in N:
        if i != j and x[i, j].solution_value() == 1:
            print(f'Ciudad {i} a ciudad {j}') #nuestro viaje

Ciudad 0 a ciudad 1
Ciudad 1 a ciudad 3
Ciudad 2 a ciudad 4
Ciudad 3 a ciudad 2
Ciudad 4 a ciudad 5
Ciudad 5 a ciudad 6
Ciudad 6 a ciudad 0
